In [2]:
import pandas as pd
import os, uuid

In [3]:
dataset = pd.read_parquet("../data/processed/dataset.parquet")

In [4]:
# on remplace les identifiants nuls par un identifiant aléatoire
dataset["identifiant_usager"] = dataset["identifiant_usager"].apply(lambda row: row if row is not None else int(str(hash(uuid.uuid4()))[:9]))

In [6]:
dataset.describe()

,jour_accident,mois_accident,annee_accident,conditions_lumieres,localisation,type_intersection,conditions_atmospheriques,type_collision,categorie_route,indice_route,...,categorie_usager,gravite_blessure,sexe_usager,annee_naissance,motif_deplacement,equipement_securite_1,equipement_securite_2,equipement_securite_3,localisation_pieton,etat_pieton
count,488934.000000,488934.000000,488934.000000,488934.000000,488934.000000,488934.000000,488934.000000,488934.000000,488934.000000,488934.000000,...,486978.000000,486978.000000,486978.000000,478439.000000,486978.000000,486978.000000,486978.000000,486978.000000,486978.000000,486978.000000
mean,15.599799,6.745413,2021.554602,1.906421,1.617284,2.068216,1.609816,3.718983,3.287755,-0.179296,...,1.329588,2.524198,1.276764,1983.152258,3.185347,1.878239,0.973258,-0.908579,-0.223700,-0.826807
std,8.749112,3.355764,1.095531,1.476384,0.486050,1.999158,1.676822,1.900646,1.273061,0.400626,...,0.610149,1.378151,0.549230,18.915494,2.771271,2.291530,3.038360,0.917200,1.226318,0.621654
min,1.000000,1.000000,2020.000000,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,1.000000,-1.000000,...,1.000000,-1.000000,-1.000000,1900.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,8.000000,4.000000,2021.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,0.000000,...,1.000000,1.000000,1.000000,1970.000000,0.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,16.000000,7.000000,2022.000000,1.000000,2.000000,1.000000,1.000000,3.000000,3.000000,0.000000,...,1.000000,3.000000,1.000000,1987.000000,4.000000,1.000000,0.000000,-1.000000,0.000000,-1.000000
75%,23.000000,10.000000,2023.000000,3.000000,2.000000,2.000000,1.000000,6.000000,4.000000,0.000000,...,2.000000,4.000000,2.000000,1998.000000,5.000000,2.000000,0.000000,-1.000000,0.000000,-1.000000
max,31.000000,12.000000,2023.000000,5.000000,2.000000,9.000000,9.000000,7.000000,9.000000,3.000000,...,3.000000,4.000000,2.000000,2023.000000,9.000000,9.000000,9.000000,9.000000,9.000000,3.000000


In [5]:
# on enlève les doublons
dataset = dataset.drop_duplicates(subset=["identifiant_usager"])

In [7]:
dataset.describe()

,jour_accident,mois_accident,annee_accident,conditions_lumieres,localisation,type_intersection,conditions_atmospheriques,type_collision,categorie_route,indice_route,...,categorie_usager,gravite_blessure,sexe_usager,annee_naissance,motif_deplacement,equipement_securite_1,equipement_securite_2,equipement_securite_3,localisation_pieton,etat_pieton
count,488934.000000,488934.000000,488934.000000,488934.000000,488934.000000,488934.000000,488934.000000,488934.000000,488934.000000,488934.000000,...,486978.000000,486978.000000,486978.000000,478439.000000,486978.000000,486978.000000,486978.000000,486978.000000,486978.000000,486978.000000
mean,15.599799,6.745413,2021.554602,1.906421,1.617284,2.068216,1.609816,3.718983,3.287755,-0.179296,...,1.329588,2.524198,1.276764,1983.152258,3.185347,1.878239,0.973258,-0.908579,-0.223700,-0.826807
std,8.749112,3.355764,1.095531,1.476384,0.486050,1.999158,1.676822,1.900646,1.273061,0.400626,...,0.610149,1.378151,0.549230,18.915494,2.771271,2.291530,3.038360,0.917200,1.226318,0.621654
min,1.000000,1.000000,2020.000000,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,1.000000,-1.000000,...,1.000000,-1.000000,-1.000000,1900.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,8.000000,4.000000,2021.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,0.000000,...,1.000000,1.000000,1.000000,1970.000000,0.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,16.000000,7.000000,2022.000000,1.000000,2.000000,1.000000,1.000000,3.000000,3.000000,0.000000,...,1.000000,3.000000,1.000000,1987.000000,4.000000,1.000000,0.000000,-1.000000,0.000000,-1.000000
75%,23.000000,10.000000,2023.000000,3.000000,2.000000,2.000000,1.000000,6.000000,4.000000,0.000000,...,2.000000,4.000000,2.000000,1998.000000,5.000000,2.000000,0.000000,-1.000000,0.000000,-1.000000
max,31.000000,12.000000,2023.000000,5.000000,2.000000,9.000000,9.000000,7.000000,9.000000,3.000000,...,3.000000,4.000000,2.000000,2023.000000,9.000000,9.000000,9.000000,9.000000,9.000000,3.000000


In [8]:
# valeurs manquantes
dataset.isna().sum()

numero_accident                   0
jour_accident                     0
mois_accident                     0
annee_accident                    0
heure_accident                    0
conditions_lumieres               0
departement                       0
commune                           0
localisation                      0
type_intersection                 0
conditions_atmospheriques         0
type_collision                    0
adresse                        8366
latitude                          0
longitude                         0
categorie_route                   0
numero_route                  57054
indice_route                      0
lettre_route                 446275
regime_circulation                0
nombre_voies                      0
presence_voie_reservee            0
profil_route                      0
numero_pr                         0
distance_pr                       0
tracage_route                     0
lartpc                       488257
larrout                     

In [62]:
dataset_without_2020["identifiant_usager"]

107194        267 638
107195        267 639
107196        267 636
107197        267 637
107198        267 634
             ...     
526839    203 720 720
526840    203 720 721
526841    203 720 717
526842    203 720 718
526843    203 720 719
Name: identifiant_usager, Length: 419650, dtype: object

In [43]:
dataset_without_2020

,numero_accident,jour_accident,mois_accident,annee_accident,heure_accident,conditions_lumieres,departement,commune,localisation,type_intersection,...,sexe_usager,annee_naissance,motif_deplacement,equipement_securite_1,equipement_securite_2,equipement_securite_3,localisation_pieton,action_pieton,etat_pieton,identifiant_usager
107194,202100000001,30,11,2021,07:32,2,30,30319,1,1,...,1.0,2000.0,1.0,0.0,9.0,-1.0,0.0,0,-1.0,267 638
107195,202100000001,30,11,2021,07:32,2,30,30319,1,1,...,1.0,1978.0,1.0,1.0,-1.0,-1.0,0.0,0,-1.0,267 639
107196,202100000002,25,9,2021,14:20,1,51,51544,1,3,...,1.0,1983.0,0.0,1.0,-1.0,-1.0,0.0,0,-1.0,267 636
107197,202100000002,25,9,2021,14:20,1,51,51544,1,3,...,1.0,1993.0,0.0,1.0,-1.0,-1.0,0.0,0,-1.0,267 637
107198,202100000003,15,7,2021,07:55,1,85,85048,2,1,...,1.0,1995.0,1.0,1.0,0.0,-1.0,0.0,0,-1.0,267 634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526839,202300054821,26,10,2023,17:20,1,973,97302,1,6,...,1.0,1971.0,1.0,2.0,0.0,-1.0,0.0,0,-1.0,203 720 720
526840,202300054821,26,10,2023,17:20,1,973,97302,1,6,...,2.0,1968.0,1.0,1.0,0.0,-1.0,0.0,0,-1.0,203 720 721
526841,202300054822,20,10,2023,16:30,1,69,69387,2,1,...,2.0,2003.0,2.0,1.0,-1.0,-1.0,-1.0,-1,-1.0,203 720 717
526842,202300054822,20,10,2023,16:30,1,69,69387,2,1,...,2.0,2002.0,1.0,1.0,-1.0,-1.0,-1.0,-1,-1.0,203 720 718


In [55]:
dataset_without_2020.loc[526839]

numero_accident                   202300054821
jour_accident                               26
mois_accident                               10
annee_accident                            2023
heure_accident                           17:20
conditions_lumieres                          1
departement                                973
commune                                  97302
localisation                                 1
type_intersection                            6
conditions_atmospheriques                    1
type_collision                               3
adresse                      ROUTE NATIONALE 1
latitude                            4,89713000
longitude                         -52,32854000
categorie_route                              2
numero_route                 ROUTE NATIONALE 1
indice_route                                 0
lettre_route                              None
regime_circulation                          -1
nombre_voies                                 2
presence_voie

In [61]:
dataset_without_2020.loc[526839].identifiant_usager

'203\xa0720\xa0720'

In [38]:
dataset_without_2020[dataset_without_2020["identifiant_usager"] == 267 638]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (211401589.py, line 1)

In [16]:
duplicates.groupby("annee_accident").describe()

jour_accident                                                 \
                       count     mean       std  min  25%   50%   75%   max   
annee_accident                                                                
2023                 37908.0  15.4436  8.713101  1.0  8.0  15.0  23.0  31.0   

               mois_accident            ... localisation_pieton       \
                       count      mean  ...                 75%  max   
annee_accident                          ...                            
2023                 37908.0  6.685528  ...                 0.0  9.0   

               etat_pieton                                               
                     count      mean       std  min  25%  50%  75%  max  
annee_accident                                                           
2023               37894.0 -0.832375  0.611928 -1.0 -1.0 -1.0 -1.0  3.0  

[1 rows x 288 columns]

37 908 doublons sur 526 844 lignes
tous concentrés sur l'année 2023